mmdetection으로 데이터학습을 통해서 모델을 만들어 봅시다.

https://mmdetection.readthedocs.io/en/latest/2_new_data_model.html



아래의 balloon dataset으로 instance segmentation을 학습해봅시다

https://github.com/matterport/Mask_RCNN/releases

데이터셋을 구글 드라이브에 압축을 풀어줍니다


위의 데이터셋을 COCO dataset으로 변환하여 진행합니다

json 파일은 vs code 등으로 Config 파일을 직접 확인하고 수정 가능합니다. 

Colab에서는 아래처럼 불러와서 확인할 수 있습니다

In [ ]:
# 라벨링 json 파일 불러오기

import json

path = '/content/drive/MyDrive/DL 202201/balloon/'

with open(path +"train/via_region_data.json", "r") as f:
    anno = json.load(f)

print(anno)

{'34020010494_e5cb88e1c4_k.jpg1115004': {'fileref': '', 'size': 1115004, 'filename': '34020010494_e5cb88e1c4_k.jpg', 'base64_img_data': '', 'file_attributes': {}, 'regions': {'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [1020, 1000, 994, 1003, 1023, 1050, 1089, 1134, 1190, 1265, 1321, 1361, 1403, 1428, 1442, 1445, 1441, 1427, 1400, 1361, 1316, 1269, 1228, 1198, 1207, 1210, 1190, 1177, 1172, 1174, 1170, 1153, 1127, 1104, 1061, 1032, 1020], 'all_points_y': [963, 899, 841, 787, 738, 700, 663, 638, 621, 619, 643, 672, 720, 765, 800, 860, 896, 942, 990, 1035, 1079, 1112, 1129, 1134, 1144, 1153, 1166, 1166, 1150, 1136, 1129, 1122, 1112, 1084, 1037, 989, 963]}, 'region_attributes': {}}}}, '25899693952_7c8b8b9edc_k.jpg814535': {'fileref': '', 'size': 814535, 'filename': '25899693952_7c8b8b9edc_k.jpg', 'base64_img_data': '', 'file_attributes': {}, 'regions': {'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [586, 510, 441, 389, 331, 268, 196, 158, 135, 156, 210, 30

In [ ]:
# 보기 좋게 출력

anno_pretty = json.dumps(anno, indent=3)
print(anno_pretty)

{
   "34020010494_e5cb88e1c4_k.jpg1115004": {
      "fileref": "",
      "size": 1115004,
      "filename": "34020010494_e5cb88e1c4_k.jpg",
      "base64_img_data": "",
      "file_attributes": {},
      "regions": {
         "0": {
            "shape_attributes": {
               "name": "polygon",
               "all_points_x": [
                  1020,
                  1000,
                  994,
                  1003,
                  1023,
                  1050,
                  1089,
                  1134,
                  1190,
                  1265,
                  1321,
                  1361,
                  1403,
                  1428,
                  1442,
                  1445,
                  1441,
                  1427,
                  1400,
                  1361,
                  1316,
                  1269,
                  1228,
                  1198,
                  1207,
                  1210,
                  1190,
                  1


COCO dataset 형식 참고

https://cocodataset.org/#format-data

```
{
    "images": [image],
    "annotations": [annotation],
    "categories": [category]
}


image = {
    "id": int,
    "width": int,
    "height": int,
    "file_name": str,
}

annotation = {
    "id": int,
    "image_id": int,
    "category_id": int,
    "segmentation": RLE or [polygon],
    "area": float,
    "bbox": [x,y,width,height],
    "iscrowd": 0 or 1,
}

categories = [{
    "id": int,
    "name": str,
    "supercategory": str,
}]
```

COCO dataset 예시

https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch


이미지 정보에 필요한 내용을 확인해보자

이미지는 PIL로 불러와보겠습니다. 

https://pillow.readthedocs.io/en/stable/

In [ ]:
from PIL import Image, ImageDraw
# 이미지는 아래의 형태로 저장 필요
'''
image = {
    "id": int,
    "width": int,
    "height": int,
    "file_name": str,
}
'''

idx = 0
for key, value in anno.items():  # 한 이미지에 대해 살펴보기
  print(key)  
  print(value)

  img = Image.open(path+'train/'+value['filename'])
  display(img)
    
  image = {'id':idx, 'width':img.width, 'height':img.height, 'file_name':value['filename'] }
  idx += 1
  print(image)
  break  # 한번만 실행해보기 위해서



라벨에 필요한 정보를 가져와 보자

In [ ]:
'''
annotation = {
    "id": int,
    "image_id": int,
    "category_id": int,
    "segmentation": RLE or [polygon],
    "area": float,
    "bbox": [x,y,width,height],
    "iscrowd": 0 or 1,
}
'''

# balloon data는 벌룬밖에 없어서 catergory_id = 0으로 입력
# polygon은 [x1,y1, x2, y2, ...] 이런 방식으로 만들어야 함
# x,y에 0.5 point 보정이 필요함 (가이드라인 참고)
# iscrowd = 0으로 입력

'\nannotation = {\n    "id": int,\n    "image_id": int,\n    "category_id": int,\n    "segmentation": RLE or [polygon],\n    "area": float,\n    "bbox": [x,y,width,height],\n    "iscrowd": 0 or 1,\n}\n'

In [ ]:
for key, value in anno.items():  # 한 이미지에 대해 살펴보기
  print(key)  
  print(value)

  img = Image.open(path+'train/'+value['filename'])
  #display(img)
  
  points_x = value['regions']['0']['shape_attributes']['all_points_x']
  points_y = value['regions']['0']['shape_attributes']['all_points_y']

  print(points_x)
  print(points_y)
  
  # PIL로 폴리곤을 그려봅시다. 
  # PIL은 (x1, y1, x2, y2, x3, y3...), 또는 ((x1, y1), (x2, y2), (x3, y3)...) 형태로 입력합니다. 

  poly = [(x, y) for x, y in zip(points_x, points_y)]  # (x,y)의 리스트 만들어 줍니다. 
  print(poly)

  draw = ImageDraw.Draw(img)
  draw.polygon(poly, fill="green") # 외곽선은 outline='green' 사용하면 됨

  display(img)
  break

PIL 폴리곤 참고

https://www.blog.pythonlibrary.org/2021/02/23/drawing-shapes-on-images-with-python-and-pillow/

In [ ]:
'''
annotation = {
    "id": int,
    "image_id": int,
    "category_id": int,
    "segmentation": RLE or [polygon],
    "area": float,
    "bbox": [x,y,width,height],
    "iscrowd": 0 or 1,
}
'''

# 한 이미지에 여러개의 라벨이 있는 경우를 고려하여 라벨을 만들어 봅시다. 

idx = 0
for key, value in anno.items():  # 한 이미지에 대해 살펴보기
  print(key)  
  print(value)

  img = Image.open(path+'train/'+value['filename'])
  
  # 여러 라벨에 대해서 반복 수행
  
  for id, shape in value['regions'].items():
    print(id)
    points_x = shape['shape_attributes']['all_points_x']
    points_y = shape['shape_attributes']['all_points_y']

    print(points_x)
    print(points_y)
    
    # 형식을 (x1, y1, x2, y2, x3, y3...) 형태로 변환합니다. 
    poly = [(x, y) for x, y in zip(points_x, points_y)]  # (x,y)의 리스트 만들어 줍니다.
    # poly = [(x + 0.5, y + 0.5) for x, y in zip(points_x, points_y)]  # tutorial에는 0.5를 더하는 것으로 되어 있음
    poly = [p for x in poly for p in x]  
    print(poly)

    # 박스 및 면적을 추출하기 위해 최대 최소를 추출
    x_min, y_min, x_max, y_max = min(points_x), min(points_y), max(points_x), max(points_y)

    # label dict 저장
    data_anno = dict(   # dict()를 사용해서 딕셔너리를 만들 수 있으면 이때는 키=값 형태로 넣어준다. 키에 "" 없이 사용 가능
                  image_id=idx,
                  id=int(id),  # 데이터에 저장된 값으로 활용
                  category_id=0,  #라벨이 balloon 한개여서 고정값
                  bbox=[x_min, y_min, x_max - x_min, y_max - y_min], # 박스 저장 [x, y, width, height]
                  area=(x_max - x_min) * (y_max - y_min), #면적
                  segmentation=[poly], # 폴리곤
                  iscrowd=0) 
                
    print(data_anno)
            
    idx += 1

    draw = ImageDraw.Draw(img)
    draw.polygon(poly, fill="green") # 외곽선은 outline='green' 사용하면 됨

  display(img)
  break


아래의 식은 list comprehension을 2개의 for문에 대해 사용한 것입니다. 

```
poly = [p for x in poly for p in x]  
```

반복문으로 풀어쓸 수 있습니다. 
```
poly = []
for x in poly:
  for p in x:
    poly.append(p)
```